In [ ]:
import json as js
import re
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import requests as req
from bs4 import BeautifulSoup

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

In [ ]:
with open("urls.json", "r") as file:
    urls = list(js.load(file))

In [ ]:
dataset = []

for index in tqdm.trange():
    requests1 = req.get(urls[index], headers=headers)
    
    soup = BeautifulSoup(requests1.content, "html")
    
    cars_info = []
    
    # Obtenir le prix
    prix = (soup.select_one("p.sc-1x0vz2r-0.lnEFFR.sc-1g3sn3w-13.czygWQ")).get_text()
    prix = re.findall(r"\d+", prix)
    prix = ''.join(prix)
    cars_info.append(prix)
    
    # Obtenir les informations dans la 1ere div
    infos = soup.select("span.sc-1x0vz2r-0.kQHNss")
    for info in infos:
        cars_info.append(info.get_text())
       
    # Obtenir les informations dans la 2eme div 
    infos1 = soup.select("span.sc-1x0vz2r-0.gSLYtF")
    for info in infos1:
        cars_info.append(info.get_text())
        
    # Obtenir les equipements de la voiture s'ils existent
    try:
        equipements = []
        for equipement in soup.select("span.sc-1x0vz2r-0.bXFCIH"):
            equipements.append(equipement.get_text())
        
        if not equipements:
            equipements = None
        else:
            equipements = ", ".join(equipements) 
            
        cars_info.append(equipements)
    
    except:
        cars_info.append(None)
    
    dataset.append(cars_info)

In [ ]:
dataset = (np.array(dataset)).T

In [ ]:
df = pd.DataFrame({
    "Price": dataset[0],
    "Year": dataset[1],
    "Transmission": dataset[2],
    "Fuel": dataset[3],
    "Type": dataset[4],
    "Sector": dataset[5],
    "Mileage": dataset[6],
    "Brand": dataset[7],
    "Model": dataset[8],
    "Number of doors": dataset[9],
    "Origin": dataset[10],
    "First owner": dataset[11],
    "Power": dataset[12],
    "Condition": dataset[13],
    "Equipment": dataset[14],
})

In [ ]:
try:
    df.to_csv("dataset.csv", index=False)
    print("Dataframe saved")
    
except IndexError as e:
    print("Dataframe didn't save")